In [ ]:
!apt-get -y install openjdk-8-jre-headless
!pip install pyspark

Reading package lists... Done
Building dependency tree       
Reading state information... Done
openjdk-8-jre-headless is already the newest version (8u312-b07-0ubuntu1~18.04).
The following packages were automatically installed and are no longer required:
  cuda-command-line-tools-10-0 cuda-command-line-tools-10-1
  cuda-command-line-tools-11-0 cuda-compiler-10-0 cuda-compiler-10-1
  cuda-compiler-11-0 cuda-cuobjdump-10-0 cuda-cuobjdump-10-1
  cuda-cuobjdump-11-0 cuda-cupti-10-0 cuda-cupti-10-1 cuda-cupti-11-0
  cuda-cupti-dev-11-0 cuda-demo-suite-10-0 cuda-demo-suite-10-1
  cuda-demo-suite-11-0 cuda-demo-suite-11-1 cuda-documentation-10-0
  cuda-documentation-10-1 cuda-documentation-11-0 cuda-documentation-11-1
  cuda-drivers cuda-drivers-510 cuda-gdb-10-0 cuda-gdb-10-1 cuda-gdb-11-0
  cuda-gpu-library-advisor-10-0 cuda-gpu-library-advisor-10-1
  cuda-libraries-10-0 cuda-libraries-10-1 cuda-libraries-11-0
  cuda-memcheck-10-0 cuda-memcheck-10-1 cuda-memcheck-11-0
  cuda-nsight-comput

In [ ]:
from pyspark import SparkContext
from pyspark import SparkConf

sparkConf = SparkConf().setAppName("hw4")
sc = SparkContext(conf = sparkConf)

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')
trainpath = '/content/gdrive/My Drive/bigdata/hw4/200[3-4].csv'
testpath = '/content/gdrive/My Drive/bigdata/hw4/2005.csv'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
selcol = [1, 3, 5, 7, 18, 25]

def handelNa(line):
    for col in selcol:
        if line[col] == 'NA':
            return False
    return True

def prepData(path):
    rawdata = sc.textFile(path)
    header = rawdata.first()

    data = rawdata.filter(lambda line: line != header) \
                  .map(lambda line: line.split(',')) \
                  .filter(lambda line: line[21] == '0') \
                  .filter(lambda line: line[23] == '0') \
                  .filter(lambda line: handelNa(line))
    # print(data.first())

    return data

In [ ]:
traindata = prepData(trainpath)
print(traindata.first())
testdata = prepData(testpath)
print(testdata.first())

['2003', '6', '29', '7', '1756', '1725', '1904', '1838', 'UA', '781', 'N228UA', '128', '133', '103', '26', '31', 'DEN', 'LAX', '862', '8', '17', '0', '', '0', '0', '26', '0', '0', '0']
['2005', '1', '28', '5', '1603', '1605', '1741', '1759', 'UA', '541', 'N935UA', '158', '174', '131', '-18', '-2', 'BOS', 'ORD', '867', '4', '23', '0', '', '0', '0', '0', '0', '0', '0']


In [ ]:
from pyspark.mllib.regression import LabeledPoint

def labelData(line):
    features = []
    for col in selcol[:-1]:
        features.append(float(line[col]))

    lbl = 0.0
    if(float(line[25]) > 0.0):
        lbl = 1.0
    return LabeledPoint(lbl, features)

In [ ]:
trainlabeldata = traindata.map(lambda line: labelData(line))
print(trainlabeldata.first())
testlabeldata = testdata.map(lambda line: labelData(line))
print(testlabeldata.first())

(1.0,[6.0,7.0,1725.0,1838.0,862.0])
(0.0,[1.0,5.0,1605.0,1759.0,867.0])


In [ ]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS

model = LogisticRegressionWithLBFGS.train(trainlabeldata)

valtraindata, validationdata = trainlabeldata.randomSplit([0.7, 0.3])
val_model = LogisticRegressionWithLBFGS.train(valtraindata)

In [ ]:
from pyspark.mllib.evaluation import BinaryClassificationMetrics

predictionAndLabels = testlabeldata.map(lambda lp: (float(model.predict(lp.features)), lp.label))
metrics = BinaryClassificationMetrics(predictionAndLabels)

print("Area under PR = %s" % metrics.areaUnderPR)
print("Area under ROC = %s" % metrics.areaUnderROC)

val_predictionAndLabels = validationdata.map(lambda lp: (float(val_model.predict(lp.features)), lp.label))
val_metrics = BinaryClassificationMetrics(val_predictionAndLabels)

print("Area under PR = %s" % val_metrics.areaUnderPR)
print("Area under ROC = %s" % val_metrics.areaUnderROC)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


Area under PR = 0.015928868937046734
Area under ROC = 0.5


/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


Area under PR = 0.014896737991571398
Area under ROC = 0.5
